! jt -r

In [7]:
from importlib import reload
reload(aux)
reload(con)
reload(dt)

<module 'src.daily_data' from 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\src\\daily_data.py'>

In [8]:
import pandas as pd
import src.daily_data as dt
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime
import src.info as info
import src.in_out as io
from src.density import Density

1. **data** klasöründeki tüm dosya yolları alınıyor

In [2]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

2. *Her bir path'de bulunan veriler **dataframe** olarak alınıyor. Her bir df, **split_df** metodu ile saatlik olarak **23** ayrı df'e bölünüyor. Bölünmüş her bir df'in **mid_price**'ı hesaplanıp yeni bir sütun olarak **df**'e ekleniyor.* **get_info** metodu her bir path için bu verileri kullanarak bir **info** örneği oluşturup döndürür. Her bir **info** örneği, aşağıdaki özelliklere sahiptir:<br>
    * Okunan dosyanın tarihi
    * Ürünün adı
    * Okunan dosyaya ait yol
    * Tüm verileri içeren dataframe
    * Saatlik olarak bölünmüş dataframe listesi


In [3]:
infolist = [info.get_info(path) for path in path_list]

3. Burada, oluşturulan infolar arasında bir **eşleştirme** yapılmaktadır. Çünkü her bir **spread** hesabının hangi veriler kullanılarak yapılacağı bilinmelidir. Eşleşen infolar, tuple çiftleri olarak tutulmakta.
  Elimizde 2018:09:06 tarihinin 6A,6B,6C ürünlerine ait verileri olsun. Bu durumda 6A-6B, 6A-6C, 6B-6C arasında 3 adet spread hesaplanacaktır. Hesaplama öncesi bu eşleştirme, **daily_data** modülünün **to_match_days** metoduyla yapılmaktadır.

In [4]:
paired_days = dt.to_match_days(infolist)

4.  **daily_datas**: GünlükVeri örneklerinin tutulduğu liste <br />
**dt.to_match_hour** metodu ile her bir eşleşmiş çiftin mid_price'ları saatlik olarak alınıp birleştiriliyor. <br />
**gunluk_veri**: **tarih**, her bir saate ait **change**,**spread**,**amplitude**,**duration**,**urun adları**...vs içeriyor <br />
**paired_hours:** 23 adet df içerir. her bir df mid_price çifti içerir

In [5]:
daily_data = []

In [9]:
for pair in paired_days:
    product = pair[0].product + "_" + pair[1].product   # urun adi
    paired_hours=dt.to_match_hour(pair[0],pair[1])
    gunluk_veri = dt.DailyData(mid_price_list=paired_hours,date=pair[0].date, name= product)
    gunluk_veri.spread_hesapla()
    daily_data.append(gunluk_veri)

haata
index:  1.0  - count:  0
index:  4.0  - count:  1.0
index:  8.0  - count:  4.0
index:  9.0  - count:  8.0
index:  13.0  - count:  9.0
index:  15.0  - count:  13.0
index:  17.0  - count:  15.0
index:  69.0  - count:  17.0
index:  70.0  - count:  69.0
index:  72.0  - count:  70.0
index:  75.0  - count:  72.0
index:  80.0  - count:  75.0
index:  84.0  - count:  80.0
index:  109.0  - count:  84.0
index:  111.0  - count:  109.0
index:  112.0  - count:  111.0
index:  115.0  - count:  112.0
index:  121.0  - count:  115.0
index:  125.0  - count:  121.0
index:  263.0  - count:  125.0
index:  473.0  - count:  263.0
index:  487.0  - count:  473.0
index:  512.0  - count:  487.0
index:  558.0  - count:  512.0
index:  694.0  - count:  558.0
index:  750.0  - count:  694.0
index:  751.0  - count:  750.0
index:  814.0  - count:  751.0
index:  935.0  - count:  814.0
index:  1004.0  - count:  935.0
index:  1117.0  - count:  1004.0
index:  1252.0  - count:  1117.0
index:  1256.0  - count:  1252.0
in

haata
index:  15.0  - count:  0
index:  17.0  - count:  15.0
index:  27.0  - count:  17.0
index:  31.0  - count:  27.0
index:  32.0  - count:  31.0
index:  36.0  - count:  32.0
index:  51.0  - count:  36.0
index:  77.0  - count:  51.0
index:  78.0  - count:  77.0
index:  80.0  - count:  78.0
index:  83.0  - count:  80.0
index:  84.0  - count:  83.0
index:  98.0  - count:  84.0
index:  105.0  - count:  98.0
index:  109.0  - count:  105.0
index:  110.0  - count:  109.0
index:  111.0  - count:  110.0
index:  114.0  - count:  111.0
index:  117.0  - count:  114.0
index:  120.0  - count:  117.0
index:  121.0  - count:  120.0
index:  122.0  - count:  121.0
index:  130.0  - count:  122.0
index:  131.0  - count:  130.0
index:  132.0  - count:  131.0
index:  133.0  - count:  132.0
index:  140.0  - count:  133.0
index:  142.0  - count:  140.0
index:  143.0  - count:  142.0
index:  166.0  - count:  143.0
index:  167.0  - count:  166.0
index:  169.0  - count:  167.0
index:  171.0  - count:  169.0
i

index:  1669.0  - count:  1664.0
index:  1670.0  - count:  1669.0
index:  1686.0  - count:  1670.0
index:  1697.0  - count:  1686.0
index:  1704.0  - count:  1697.0
index:  1707.0  - count:  1704.0
index:  1712.0  - count:  1707.0
index:  1722.0  - count:  1712.0
index:  1729.0  - count:  1722.0
index:  1762.0  - count:  1729.0
index:  1763.0  - count:  1762.0
index:  1769.0  - count:  1763.0
index:  1771.0  - count:  1769.0
index:  1784.0  - count:  1771.0
index:  1790.0  - count:  1784.0
index:  1821.0  - count:  1790.0
index:  1825.0  - count:  1821.0
index:  1829.0  - count:  1825.0
index:  1830.0  - count:  1829.0
index:  1834.0  - count:  1830.0
index:  1846.0  - count:  1834.0
index:  1860.0  - count:  1846.0
index:  1867.0  - count:  1860.0
index:  1892.0  - count:  1867.0
index:  1893.0  - count:  1892.0
index:  1906.0  - count:  1893.0
index:  1924.0  - count:  1906.0
index:  1930.0  - count:  1924.0
index:  1969.0  - count:  1930.0
index:  1970.0  - count:  1969.0
index:  19

index:  1981.0  - count:  1980.0
index:  1984.0  - count:  1981.0
index:  1985.0  - count:  1984.0
index:  1991.0  - count:  1985.0
index:  1993.0  - count:  1991.0
index:  2001.0  - count:  1993.0
index:  2007.0  - count:  2001.0
index:  2008.0  - count:  2007.0
index:  2024.0  - count:  2008.0
index:  2029.0  - count:  2024.0
index:  2036.0  - count:  2029.0
index:  2049.0  - count:  2036.0
index:  2055.0  - count:  2049.0
index:  2059.0  - count:  2055.0
index:  2060.0  - count:  2059.0
index:  2063.0  - count:  2060.0
index:  2068.0  - count:  2063.0
index:  2074.0  - count:  2068.0
index:  2110.0  - count:  2074.0
index:  2113.0  - count:  2110.0
index:  2127.0  - count:  2113.0
index:  2128.0  - count:  2127.0
index:  2129.0  - count:  2128.0
index:  2131.0  - count:  2129.0
index:  2132.0  - count:  2131.0
index:  2133.0  - count:  2132.0
index:  2135.0  - count:  2133.0
index:  2141.0  - count:  2135.0
index:  2142.0  - count:  2141.0
index:  2144.0  - count:  2142.0
index:  21

haata
index:  1.0  - count:  0
index:  3.0  - count:  1.0
index:  18.0  - count:  3.0
index:  24.0  - count:  18.0
index:  55.0  - count:  24.0
index:  56.0  - count:  55.0
index:  129.0  - count:  56.0
index:  132.0  - count:  129.0
index:  133.0  - count:  132.0
index:  137.0  - count:  133.0
index:  192.0  - count:  137.0
index:  195.0  - count:  192.0
index:  197.0  - count:  195.0
index:  198.0  - count:  197.0
index:  200.0  - count:  198.0
index:  204.0  - count:  200.0
index:  207.0  - count:  204.0
index:  238.0  - count:  207.0
index:  290.0  - count:  238.0
index:  297.0  - count:  290.0
index:  301.0  - count:  297.0
index:  302.0  - count:  301.0
index:  340.0  - count:  302.0
index:  359.0  - count:  340.0
index:  372.0  - count:  359.0
index:  373.0  - count:  372.0
index:  379.0  - count:  373.0
index:  380.0  - count:  379.0
index:  427.0  - count:  380.0
index:  485.0  - count:  427.0
index:  511.0  - count:  485.0
index:  530.0  - count:  511.0
index:  538.0  - count

...

index:  2047.0  - count:  2046.0
index:  2049.0  - count:  2047.0
index:  2061.0  - count:  2049.0
index:  2062.0  - count:  2061.0
index:  2064.0  - count:  2062.0
index:  2077.0  - count:  2064.0
index:  2079.0  - count:  2077.0
index:  2083.0  - count:  2079.0
index:  2085.0  - count:  2083.0
index:  2088.0  - count:  2085.0
index:  2089.0  - count:  2088.0
index:  2093.0  - count:  2089.0
index:  2096.0  - count:  2093.0
index:  2103.0  - count:  2096.0
index:  2137.0  - count:  2103.0
index:  2156.0  - count:  2137.0
index:  2166.0  - count:  2156.0
index:  2185.0  - count:  2166.0
index:  2187.0  - count:  2185.0
index:  2194.0  - count:  2187.0
index:  2198.0  - count:  2194.0
index:  2200.0  - count:  2198.0
index:  2201.0  - count:  2200.0
index:  2203.0  - count:  2201.0
index:  2239.0  - count:  2203.0
index:  2242.0  - count:  2239.0
index:  2260.0  - count:  2242.0
index:  2263.0  - count:  2260.0
index:  2264.0  - count:  2263.0
index:  2288.0  - count:  2264.0
index:  22

index:  1140.0  - count:  1139.0
index:  1142.0  - count:  1140.0
index:  1175.0  - count:  1142.0
index:  1178.0  - count:  1175.0
index:  1203.0  - count:  1178.0
index:  1205.0  - count:  1203.0
index:  1207.0  - count:  1205.0
index:  1212.0  - count:  1207.0
index:  1222.0  - count:  1212.0
index:  1226.0  - count:  1222.0
index:  1243.0  - count:  1226.0
index:  1252.0  - count:  1243.0
index:  1253.0  - count:  1252.0
index:  1255.0  - count:  1253.0
index:  1271.0  - count:  1255.0
index:  1278.0  - count:  1271.0
index:  1285.0  - count:  1278.0
index:  1286.0  - count:  1285.0
index:  1294.0  - count:  1286.0
index:  1300.0  - count:  1294.0
index:  1304.0  - count:  1300.0
index:  1319.0  - count:  1304.0
index:  1322.0  - count:  1319.0
index:  1334.0  - count:  1322.0
index:  1337.0  - count:  1334.0
index:  1353.0  - count:  1337.0
index:  1358.0  - count:  1353.0
index:  1367.0  - count:  1358.0
index:  1370.0  - count:  1367.0
index:  1379.0  - count:  1370.0
index:  13

haata
index:  1.0  - count:  0
index:  9.0  - count:  1.0
index:  24.0  - count:  9.0
index:  41.0  - count:  24.0
index:  65.0  - count:  41.0
index:  71.0  - count:  65.0
index:  74.0  - count:  71.0
index:  76.0  - count:  74.0
index:  113.0  - count:  76.0
index:  136.0  - count:  113.0
index:  138.0  - count:  136.0
index:  139.0  - count:  138.0
index:  142.0  - count:  139.0
index:  143.0  - count:  142.0
index:  144.0  - count:  143.0
index:  165.0  - count:  144.0
index:  167.0  - count:  165.0
index:  198.0  - count:  167.0
index:  199.0  - count:  198.0
index:  217.0  - count:  199.0
index:  219.0  - count:  217.0
index:  220.0  - count:  219.0
index:  222.0  - count:  220.0
index:  223.0  - count:  222.0
index:  233.0  - count:  223.0
index:  234.0  - count:  233.0
index:  241.0  - count:  234.0
index:  242.0  - count:  241.0
index:  243.0  - count:  242.0
index:  283.0  - count:  243.0
index:  284.0  - count:  283.0
index:  289.0  - count:  284.0
index:  291.0  - count:  2

index:  3123.0  - count:  3111.0
index:  3141.0  - count:  3123.0
index:  3143.0  - count:  3141.0
index:  3173.0  - count:  3143.0
index:  3179.0  - count:  3173.0
index:  3180.0  - count:  3179.0
index:  3210.0  - count:  3180.0
index:  3221.0  - count:  3210.0
index:  3244.0  - count:  3221.0
index:  3245.0  - count:  3244.0
index:  3272.0  - count:  3245.0
index:  3279.0  - count:  3272.0
index:  3299.0  - count:  3279.0
index:  3300.0  - count:  3299.0
index:  3365.0  - count:  3300.0
index:  3373.0  - count:  3365.0
index:  3378.0  - count:  3373.0
index:  3381.0  - count:  3378.0
index:  3383.0  - count:  3381.0
index:  3401.0  - count:  3383.0
index:  3412.0  - count:  3401.0
index:  3420.0  - count:  3412.0
index:  3427.0  - count:  3420.0
index:  3469.0  - count:  3427.0
index:  3500.0  - count:  3469.0
index:  3506.0  - count:  3500.0
index:  3540.0  - count:  3506.0
index:  3542.0  - count:  3540.0
index:  3596.0  - count:  3542.0
index:  3597.0  - count:  3596.0
haata
inde

haata
index:  1.0  - count:  0
index:  5.0  - count:  1.0
index:  7.0  - count:  5.0
index:  26.0  - count:  7.0
index:  27.0  - count:  26.0
index:  34.0  - count:  27.0
index:  42.0  - count:  34.0
index:  43.0  - count:  42.0
index:  56.0  - count:  43.0
index:  77.0  - count:  56.0
index:  88.0  - count:  77.0
index:  91.0  - count:  88.0
index:  95.0  - count:  91.0
index:  142.0  - count:  95.0
index:  192.0  - count:  142.0
index:  207.0  - count:  192.0
index:  215.0  - count:  207.0
index:  247.0  - count:  215.0
index:  255.0  - count:  247.0
index:  269.0  - count:  255.0
index:  270.0  - count:  269.0
index:  275.0  - count:  270.0
index:  276.0  - count:  275.0
index:  277.0  - count:  276.0
index:  279.0  - count:  277.0
index:  280.0  - count:  279.0
index:  288.0  - count:  280.0
index:  294.0  - count:  288.0
index:  296.0  - count:  294.0
index:  304.0  - count:  296.0
index:  307.0  - count:  304.0
index:  311.0  - count:  307.0
index:  315.0  - count:  311.0
index: 

5. Her bir **DailyData** örneği ürünlere göre ayrı ayrı gruplandırılıyor. <br>
    (['**6AU8_6BU8**': [gunluk veriler...], '**6AU8_6CU8**':[gunluk veriler...], '**6BU8_6CU8**':[gunluk veriler...]])

In [10]:
products={}

In [12]:
for gunluk in daily_data:
    if gunluk.name in products:
        products[gunluk.name].append(gunluk)
    else:
        products[gunluk.name] = [gunluk]

**combined_days :**  key-value olarak ürün adları ve verileri içerir <br />
**combined_days_detail :** key-value olarak ürün adları ve verilere ait istatistiksel bilgileri içerir <br />
**combined_days_con :** key-value olarak ürün adları ve median koşuluna göre yeniden düzenlenmiş verileri içerir <br />
* 6BU8_6CU8'e ait verilere erişmek için:<br />
 ```python
combined_days_detail['6BU8_6CU8']
```
* 6BU8_6CU8'e ait 18:00:00-19:00:00 saatleri arası verilere erişmek için: <br />
 ```python
combined_days['6BU8_6CU8'].loc[time(18)]
```

In [13]:
combined_days = {}
combined_days_detail = {}
combined_days_con = {}
combined_days_con_detail = {}

In [14]:
stop=False
i=0
while products:
    print(i)
    urun = products.popitem()
    df = dt.get_all_data(urun[1]).dropna()
    #con_df = con.scan(df)
    combined_days[urun[0]] = df
#     combined_days_con[urun[0]] = con_df
    combined_days_detail[urun[0]] = aux.get_detail(df)
#     combined_days_con_detail[urun[0]] = aux.get_detail(con_df)

0


Hesaplanmış ürün adları ve saat bilgisi alınıyor

In [15]:
all_products = list(combined_days.keys())

In [16]:
hour_series = pd.date_range('2018-01-01-18', periods=23, freq='H').time

all_density:
```json
{"6BU8_6CU8": {"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 "6AU8_6CU8":{"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 ....}
```

In [17]:
all_density= {}

In [18]:
for product in all_products:
    densities = {}
    for hour in hour_series:
        df = combined_days[product].loc[hour]
        densities[hour]= Density(df)
    all_density[product] = densities

--------------------------------

# Example

In [22]:
data_of_the_eighteenth_clock = all_density['6AU8_6BU8'][time(18)]

data_of_the_eighteenth_clock.data

data_of_the_eighteenth_clock.density_data

In [23]:
data_of_the_eighteenth_clock.draw()